In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")


In [2]:
import requests
from io import BytesIO
from openai import OpenAI

client = OpenAI()

def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    return result.id

# Replace with your own file path or URL
file_id = create_file(client, "https://cdn.openai.com/API/docs/deep_research_blog.pdf")

file-BW8h92v4RKLxiBcaJGJ4ix


In [3]:
file_id

'file-BW8h92v4RKLxiBcaJGJ4ix'

In [4]:
vector_store = client.vector_stores.create(
    name="knowledge_base"
)
print(vector_store.id)

vs_67d1bdb576788191841b2bd18b054d42


In [5]:
result=client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)
print(result)

VectorStoreFile(id='file-BW8h92v4RKLxiBcaJGJ4ix', created_at=1741798865, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_67d1bdb576788191841b2bd18b054d42', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))


In [7]:
from agents import Agent, FileSearchTool, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool(),
         FileSearchTool(
            max_num_results=3,
            vector_store_ids=[vector_store.id],
        ),
    ],
)

async def main():
    result = await Runner.run(agent, "what is prerequisite for creating new knowledge")
    print(result.final_output)

In [8]:
if __name__=="__main__":
    await main()

The prerequisite for creating new knowledge is the ability to synthesize existing knowledge. This involves reasoning, analyzing, and consolidating information from various sources, which is crucial for developing insights that lead to the creation of novel research.
